# Equal-Weight S&P 500 Index Fund

## Introduction & Library Imports

The S&P 500 is the world's most popular stock market index. The largest fund that is benchmarked to this index is the SPDR® S&P 500® ETF Trust. It has more than US$250 billion of assets under management.

The goal of this section of the course is to create a Python script that will accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

## Library Imports

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [9]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from tqdm import tqdm

## Importing Our List of Stocks

The next thing we need to do is import the constituents of the S&P 500.

These constituents change over time, so in an ideal world you would connect directly to the index provider (Standard & Poor's) and pull their real-time constituents on a regular basis.

Paying for access to the index provider's API is outside of the scope of this course. 

There's a static version of the S&P 500 constituents available here. [Click this link to download them now](https://drive.google.com/file/d/1ZJSpbY69DVckVZlO9cC6KkgfSufybcHN/view?usp=sharing). Move this file into the `starter-files` folder so it can be accessed by other files in that directory.

Now it's time to import these stocks to our Jupyter Notebook file.

In [10]:
stocks = pd.read_csv("sp_500_stocks.csv")
stocks.head(3)

,Ticker
0,A
1,AAL
2,AAP


## Acquiring an API Token

Now it's time to import our IEX Cloud API token. This is the data provider that we will be using throughout this course.

API tokens (and other sensitive information) should be stored in a `secrets.py` file that doesn't get pushed to your local Git repository. We'll be using a sandbox API token in this course, which means that the data we'll use is randomly-generated and (more importantly) has no cost associated with it.

[Click here](http://nickmccullum.com/algorithmic-trading-python/secrets.py) to download your `secrets.py` file. Move the file into the same directory as this Jupyter Notebook before proceeding.

In [51]:
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

Now it's time to structure our API calls to IEX cloud. 

We need the following information from the API:

* Market capitalization for each stock
* Price of each stock



In [12]:
symbol = 'AAPL'
# https:/cloud.iexapis.com/
api_url = f"https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}" 
data = requests.get(api_url)
assert(data.status_code == 200)
data = data.json()

## Parsing Our API Call

The API call that we executed in the last code block contains all of the information required to build our equal-weight S&P 500 strategy. 

With that said, the data isn't in a proper format yet. We need to parse it first.

In [13]:
price = data["latestPrice"]
market_cap = data["marketCap"]

## Using Batch API Calls to Improve Performance

Batch API calls are one of the easiest ways to improve the performance of your code.

This is because HTTP requests are typically one of the slowest components of a script.

Also, API providers will often give you discounted rates for using batch API calls since they are easier for the API provider to respond to.

IEX Cloud limits their batch API calls to 100 tickers per request. Still, this reduces the number of API calls we'll make in this section from 500 to 5 - huge improvement! In this section, we'll split our list of stocks into groups of 100 and then make a batch API call for each group.

In [22]:
def chunks_generator(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [60]:
symbol_groups = list(chunks_generator(stocks["Ticker"], 100))
symbol_strings = [",".join(symbol_groups[i]) for i in range(len(symbol_groups))]
df = pd.DataFrame(columns=cols)
for symbol_string in symbol_strings:
    batch_request_url = f"https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote&token={IEX_CLOUD_API_TOKEN}"
    data = requests.get(batch_request_url)
    assert(data.status_code == 200)
    data = data.json()
    for symbol in symbol_string.split(","):
        df = df.append(pd.Series([
            symbol, 
            data[symbol]['quote']['latestPrice'],
            data[symbol]['quote']['marketCap'],
            pd.NA]
            , index=cols), ignore_index=True)

In [62]:
df.head()

,ticker,stock_price,market_cap,num_of_shares_to_buy
0,A,119.42,37274108112,<NA>
1,AAL,16.30,10007399173,<NA>
2,AAP,162.05,11079106627,<NA>
3,AAPL,133.07,2281705540198,<NA>
4,ABBV,106.38,187378071107,<NA>


## Calculating the Number of Shares to Buy

As you can see in the DataFrame above, we stil haven't calculated the number of shares of each stock to buy.

In [83]:
portfolio_size=1_000_000
position_size = portfolio_size/len(df.index)
print(f"Maximum position size in each stock: {round(position_size,2)}")
for i in range(len(df.index)):
    df.loc[i, "num_of_shares_to_buy"] = math.floor(position_size/df.loc[i, "stock_price"])

Maximum position size in each stock: 1980.2


In [85]:
df.head()

,ticker,stock_price,market_cap,num_of_shares_to_buy
0,A,119.42,37274108112,16
1,AAL,16.30,10007399173,121
2,AAP,162.05,11079106627,12
3,AAPL,133.07,2281705540198,14
4,ABBV,106.38,187378071107,18


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

### Initializing our XlsxWriter Object

In [154]:
writer = pd.ExcelWriter("recommended_trades.xlsx", engine="xlsxwriter")
df.to_excel(writer, "Trades", index=False)

### Creating the Formats We'll Need For Our `.xlsx` File

Formats include colors, fonts, and also symbols like `%` and `$`. We'll need four main formats for our Excel document:
* String format for tickers
* \\$XX.XX format for stock prices
* \\$XX,XXX format for market capitalization
* Integer format for the number of shares to purchase

In [155]:
background_color = '#0a0a23'
font_color = '#fffff'

string_format = writer.book.add_format({"font_color": font_color, "bg_color": background_color, "border": 1})
euro_format = writer.book.add_format({"font_color": font_color, "bg_color": background_color, "border": 1, "num_format": "0,00 €"})
integer_format = writer.book.add_format({"font_color": font_color, "bg_color": background_color, "border": 1, "num_format": "0"})

### Applying the Formats to the Columns of Our `.xlsx` File

We can use the `set_column` method applied to the `writer.sheets['Recommended Trades']` object to apply formats to specific columns of our spreadsheets.

Here's an example:

```python
writer.sheets['Recommended Trades'].set_column('B:B', #This tells the method to apply the format to column B
                     18, #This tells the method to apply a column width of 18 pixels
                     string_template #This applies the format 'string_template' to the column
                    )
```

In [157]:
column_format = {
    "A": ["ticker", string_format],
    "B": ["stock_price", euro_format],
    "C": ["market_cap", euro_format],
    "D": ["num_of_shares_to_buy", integer_format]
}

for col in column_format.keys():
    writer.sheets["Trades"].set_column(f"{col}:{col}", 18, column_format[col][1])
    writer.sheets["Trades"].write(f"{col}1", column_format[col][0], column_format[col][1])
writer.save()